In [1]:
from allosaurus.app import read_recognizer
from praatio import textgrid
from itertools import zip_longest
from hashlib import sha1
from pathlib import Path
from tqdm.auto import tqdm
import joblib

## Allosaurus branch
`https://github.com/seantyh/allosaurus-emit/tree/5366a7`

## Data dependencies

In [2]:
wav_paths = list(Path("data").glob("*.wav"))
for path_x in wav_paths:
    h = sha1()
    h.update(path_x.read_bytes())
    print(path_x, h.hexdigest()[:6], sep="\t")

data\劉櫂豪-127903.wav	a2caca
data\許淑華-127889.wav	b76b15
data\趙正宇-127849.wav	d8bb3e
data\陳椒華-127940.wav	0071cb
data\陳瑩-128068.wav	34d27f
data\黃國書-127909.wav	2a7fac


## Setting-up environment

In [3]:
model = read_recognizer()

In [4]:
wav_paths = list(Path("data").glob("*.wav"))
out_dir = Path("data/emit2")
out_dir.mkdir(parents=True, exist_ok=True)
wav_paths

[WindowsPath('data/劉櫂豪-127903.wav'),
 WindowsPath('data/許淑華-127889.wav'),
 WindowsPath('data/趙正宇-127849.wav'),
 WindowsPath('data/陳椒華-127940.wav'),
 WindowsPath('data/陳瑩-128068.wav'),
 WindowsPath('data/黃國書-127909.wav')]

## Generate emit frames & TextGrids

In [5]:
emit_val = 2
for wav_path_x in tqdm(wav_paths, ):
    frames = model.recognize(wav_path_x, lang_id="cmn", emit=emit_val)
    for x, y in zip_longest(frames, frames[1:]):
        if y is None:
            end_time = x.offset_s+.045
        else:
            end_time = y.offset_s
        x.endtime_s = end_time

    tg = textgrid.Textgrid()
    frame_entries = [(x.offset_s, y.offset_s, x.phone_token) for x, y in zip(frames, frames[1:])]
    frame_tier = textgrid.IntervalTier('allosaurus', frame_entries[::1], 0, frames[-1].offset_s+.045)
    tg_name = wav_path_x.name.replace(".wav", f".allo.emit{emit_val}.TextGrid")
    tg_path_x = out_dir / tg_name
    tg.addTier(frame_tier)
    tg.save(tg_path_x, format="short_textgrid", includeBlankSpaces=False)
    
    emit_name = "emit_frames_" + wav_path_x.name.replace(".wav", ".emit2.pkl")
    emit_path_x = out_dir / emit_name
    joblib.dump(frames, emit_path_x)

  0%|          | 0/6 [00:00<?, ?it/s]

## Output hashes

```
data\emit2\emit_frames_劉櫂豪-127903.emit2.pkl	b9b100
data\emit2\emit_frames_許淑華-127889.emit2.pkl	27dd8b
data\emit2\emit_frames_趙正宇-127849.emit2.pkl	b43c78
data\emit2\emit_frames_陳椒華-127940.emit2.pkl	d714a9
data\emit2\emit_frames_陳瑩-128068.emit2.pkl	aef224
data\emit2\emit_frames_黃國書-127909.emit2.pkl	5fd6d6
data\emit2\劉櫂豪-127903.allo.emit2.TextGrid	f37c11
data\emit2\許淑華-127889.allo.emit2.TextGrid	b068be
data\emit2\趙正宇-127849.allo.emit2.TextGrid	37a2fc
data\emit2\陳椒華-127940.allo.emit2.TextGrid	c17801
data\emit2\陳瑩-128068.allo.emit2.TextGrid	999111
data\emit2\黃國書-127909.allo.emit2.TextGrid	008788
----
Overall Hash:  57438b
```

In [7]:
hash_buf = []
for path_x in out_dir.iterdir():
    h = sha1()
    h.update(path_x.read_bytes())
    msg = h.hexdigest()
    hash_buf.append(f"{str(path_x)}\t{msg[:6]}")
    print(path_x, msg[:6], sep="\t")

h = sha1()
h.update("".join(hash_buf).encode())
print("----\nOverall Hash: ", h.hexdigest()[:6])

data\emit2\emit_frames_劉櫂豪-127903.emit2.pkl	b9b100
data\emit2\emit_frames_許淑華-127889.emit2.pkl	27dd8b
data\emit2\emit_frames_趙正宇-127849.emit2.pkl	b43c78
data\emit2\emit_frames_陳椒華-127940.emit2.pkl	d714a9
data\emit2\emit_frames_陳瑩-128068.emit2.pkl	aef224
data\emit2\emit_frames_黃國書-127909.emit2.pkl	5fd6d6
data\emit2\劉櫂豪-127903.allo.emit2.TextGrid	f37c11
data\emit2\許淑華-127889.allo.emit2.TextGrid	b068be
data\emit2\趙正宇-127849.allo.emit2.TextGrid	37a2fc
data\emit2\陳椒華-127940.allo.emit2.TextGrid	c17801
data\emit2\陳瑩-128068.allo.emit2.TextGrid	999111
data\emit2\黃國書-127909.allo.emit2.TextGrid	008788
----
Overall Hash:  57438b
